In [1]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\users\crie072\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

In [3]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200828


In [4]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

ディレクトリ

In [5]:
data_dir = Path(r"C:\Users\crie072\Documents\21_Online\オンライン入会状況レポート")
data_dir2 = Path(r"C:\Users\crie072\Documents\21_Online")

In [123]:
#input
input_file = "【C116】新規入会者_6月末時点"

input_file2="デジガレアフィリエイト申込累積"

code_list = "code-todoufuken"

#out_put
output_file = "新規入会属性"
output_file1 = "オンライン入会者の利用状況調査レポート作成用"
output_file2="オンライン入会者の利用状況調査レポート作成用_アフィリエイト別"

In [124]:
#CSCからの月次入会者リストを読込
df = pd.read_excel(data_dir /r"input"/f"{input_file}.xlsx",encoding='shift-jis')

In [125]:
df.head()

,実績月,直近６ケ月買物利用金額,直近６ケ月全体利用金額,年令,性別,都道府県名,顧客状況コ－ド,マイペ設定,DM希望店コード,カード券種名,カ－ド加入日,初回利用日,あとからリボ申込有無コード,Weｂ明細有無コード,解約,入会チャネル,入会申込番号
0,202002,0,0,82,女性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋お得意様ゴールド（VISA）,20191219,0,0,0,0,郵送,155051921627812
1,202002,0,0,66,男性,大阪府,0,0,Ｄ_心斎橋店,大丸松坂屋お得意様ゴールド（マスター）,20200120,0,0,0,0,郵送,155051921627812
2,202002,0,0,59,女性,大阪府,0,0,Ｄ_梅田店,大丸松坂屋マスターカード,20200116,0,0,1,0,オンライン,155051921627812
3,202002,36,36,70,男性,兵庫県,0,0,Ｄ_梅田店,大丸松坂屋お得意様ゴールド（マスター）,20200120,0,0,0,0,郵送,155051921627812
4,202002,0,0,86,女性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAカード,20200213,0,0,0,0,店頭タブレット,155051921627812


In [126]:
#重複削除
df_dup = df.drop_duplicates(subset=['入会申込番号','実績月'])
#df_dup = df.copy()
df_dup.shape[0]

44945

In [127]:
df_dup.head(2)

,実績月,直近６ケ月買物利用金額,直近６ケ月全体利用金額,年令,性別,都道府県名,顧客状況コ－ド,マイペ設定,DM希望店コード,カード券種名,カ－ド加入日,初回利用日,あとからリボ申込有無コード,Weｂ明細有無コード,解約,入会チャネル,入会申込番号
0,202002,0,0,82,女性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋お得意様ゴールド（VISA）,20191219,0,0,0,0,郵送,155051921627812
1,202002,0,0,66,男性,大阪府,0,0,Ｄ_心斎橋店,大丸松坂屋お得意様ゴールド（マスター）,20200120,0,0,0,0,郵送,155051921627812


In [128]:
#都道府県コードリスト
code_list = pd.read_excel(data_dir /r"input"/f"{code_list}.xlsx",encoding='shift-jis')
code_list.head()

,prefecture,prefecture2
0,北海道,01_北海道
1,青森県,02_青森県
2,岩手県,03_岩手県
3,宮城県,04_宮城県
4,秋田県,05_秋田県


In [129]:
_df_dup = df_dup.copy() #エラー回避
_df_dup['年代'] = _df_dup['年令']//10 * 10
df_dup = _df_dup.copy()

In [130]:
df_dup = df_dup.rename(columns={'実績月':'cp',
                        '入会申込番号':'applino',
                        '直近６ケ月全体利用金額':'credit_amt_6m',
                        '入会チャネル':'channel',
                        '解約':'kaiyaku_f',
                        '初回利用日':'first_use_dt',
                        'カ－ド加入日':'kanyu_dt',
                        'カード券種名':'card',
                        '性別':'gender',
                        '年代':'age',
                        '都道府県名':'prefecture',
                        'Weｂ明細有無コード':'web_f'})
df_dup = df_dup.drop(columns={'直近６ケ月買物利用金額','顧客状況コ－ド','マイペ設定','DM希望店コード','年令',
                     'あとからリボ申込有無コード'})
df_dup.head(2)

,cp,credit_amt_6m,gender,prefecture,card,kanyu_dt,first_use_dt,web_f,kaiyaku_f,channel,applino,age
0,202002,0,女性,兵庫県,大丸松坂屋お得意様ゴールド（VISA）,20191219,0,0,0,郵送,155051921627812,80
1,202002,0,男性,大阪府,大丸松坂屋お得意様ゴールド（マスター）,20200120,0,0,0,郵送,155051921627812,60


In [131]:
df_dup = pd.merge(df_dup,code_list, on =['prefecture'], how = 'left')
df_dup.head()

,cp,credit_amt_6m,gender,prefecture,card,kanyu_dt,first_use_dt,web_f,kaiyaku_f,channel,applino,age,prefecture2
0,202002,0,女性,兵庫県,大丸松坂屋お得意様ゴールド（VISA）,20191219,0,0,0,郵送,155051921627812,80,28_兵庫県
1,202002,0,男性,大阪府,大丸松坂屋お得意様ゴールド（マスター）,20200120,0,0,0,郵送,155051921627812,60,27_大阪府
2,202002,0,女性,大阪府,大丸松坂屋マスターカード,20200116,0,1,0,オンライン,155051921627812,50,27_大阪府
3,202002,36,男性,兵庫県,大丸松坂屋お得意様ゴールド（マスター）,20200120,0,0,0,郵送,155051921627812,70,28_兵庫県
4,202002,0,女性,兵庫県,大丸松坂屋VISAカード,20200213,0,0,0,店頭タブレット,155051921627812,80,28_兵庫県


In [132]:
print(df_dup.dtypes)

cp                int64
credit_amt_6m     int64
gender           object
prefecture       object
card             object
kanyu_dt          int64
first_use_dt      int64
web_f             int64
kaiyaku_f         int64
channel          object
applino           int64
age               int64
prefecture2      object
dtype: object


In [133]:
df_dup['cp_dt'] = df_dup['cp']*100+1
df_dup['cp_dt'] = pd.to_datetime(df_dup['cp_dt'], format='%Y%m%d')

In [134]:
df_dup['kanyu_dt']=pd.to_datetime(df_dup['kanyu_dt'], format='%Y%m%d')
df_dup['kanyu_year'] = df_dup['kanyu_dt'].dt.year
df_dup['kanyu_month'] = df_dup['kanyu_dt'].dt.month

df_dup['kanyu_yymm'] = df_dup[['kanyu_year', 'kanyu_month']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)

In [135]:
#経過月数
df_dup['mob']=(df_dup['cp_dt'].dt.year - df_dup['kanyu_dt'].dt.year) * 12 + df_dup['cp_dt'].dt.month - df_dup['kanyu_dt'].dt.month
df_dup.head(2)

,cp,credit_amt_6m,gender,prefecture,card,kanyu_dt,first_use_dt,web_f,kaiyaku_f,channel,applino,age,prefecture2,cp_dt,kanyu_year,kanyu_month,kanyu_yymm,mob
0,202002,0,女性,兵庫県,大丸松坂屋お得意様ゴールド（VISA）,2019-12-19,0,0,0,郵送,155051921627812,80,28_兵庫県,2020-02-01,2019,12,2019年12月,2
1,202002,0,男性,大阪府,大丸松坂屋お得意様ゴールド（マスター）,2020-01-20,0,0,0,郵送,155051921627812,60,27_大阪府,2020-02-01,2020,1,2020年1月,1


In [136]:
print(df_dup.dtypes)

cp                        int64
credit_amt_6m             int64
gender                   object
prefecture               object
card                     object
kanyu_dt         datetime64[ns]
first_use_dt              int64
web_f                     int64
kaiyaku_f                 int64
channel                  object
applino                   int64
age                       int64
prefecture2              object
cp_dt            datetime64[ns]
kanyu_year                int64
kanyu_month               int64
kanyu_yymm               object
mob                       int64
dtype: object


In [137]:
print(df_dup['card'].unique())

['大丸松坂屋お得意様ゴールド（VISA）' '大丸松坂屋お得意様ゴールド（マスター）' '大丸松坂屋マスターカード' '大丸松坂屋VISAカード'
 'さくらパンダVISAカード' '大丸松坂屋マスターゴールド' '大丸松坂屋VISAゴールド' '大丸松坂屋マスターカード（大阪学校生協）'
 '大丸松坂屋従業員マスターカード' '大丸松坂屋お得意様VISA（高知）' '大丸松坂屋お得意様マスター（下関）'
 'さくらパンダVISAカード（学生）' '大丸松坂屋VISAカード（学生）' '大丸松坂屋マスターカード（学生）'
 '大丸松坂屋お得意様VISA（下関）' '大丸松坂屋従業員VISAカード' '大丸松坂屋VISAカード（ヒューリック）'
 '大丸松坂屋お得意様マスター（高知）']


In [118]:
q = """
        SELECT
        cp
        ,channel
        ,kanyu_year
        ,kanyu_month
        ,kanyu_yymm
        ,age
        ,gender
        ,web_f
        ,prefecture2
        ,count(applino) as sum_no_account
        from 
            df_dup
        where 
            mob = 0
        group by
            1,2,3,4,5,6,7,8,9
        """
df_new = sqldf(q, locals())
df_new.head()

,cp,channel,kanyu_year,kanyu_month,kanyu_yymm,age,gender,web_f,prefecture2,sum_no_account
0,202006,オンライン,2020,6,2020年6月,10,女性,1,13_東京都,1
1,202006,オンライン,2020,6,2020年6月,10,女性,1,27_大阪府,1
2,202006,オンライン,2020,6,2020年6月,10,女性,1,28_兵庫県,1
3,202006,オンライン,2020,6,2020年6月,10,女性,1,35_山口県,1
4,202006,オンライン,2020,6,2020年6月,10,男性,1,28_兵庫県,1


In [119]:
#CSV出力（新規属性）
#df_new.to_csv(data_dir /r"Output"/f"{output_file}_{input_file}.csv", index=None,header=True,encoding='shift-jis')

In [138]:
q = """
        SELECT
        cp
        ,channel
        ,case
            when card like '%お得意様%' then '外商'
            when card like '%ゴールド%' then 'ゴールド'
            else '一般'
        end as card_cate
        ,kanyu_year
        ,kanyu_month
        ,kanyu_yymm
        ,mob
        ,count(applino) as sum_no_account
        ,sum(case when first_use_dt > 0 then 1 else 0 end) as sum_no_1st_use
        ,sum(case when kaiyaku_f = 1 then 1 else 0 end) as sum_no_kaiyaku
        ,sum(credit_amt_6m) as sum_credit_amt6m
        from 
            df_dup
        group by
            1,2,3,4,5,6,7
        """
df_sql = sqldf(q, locals())
df_sql.head()

,cp,channel,card_cate,kanyu_year,kanyu_month,kanyu_yymm,mob,sum_no_account,sum_no_1st_use,sum_no_kaiyaku,sum_credit_amt6m
0,202001,オンライン,ゴールド,2019,12,2019年12月,1,34,29,1,8145
1,202001,オンライン,ゴールド,2020,1,2020年1月,0,29,13,0,1070
2,202001,オンライン,一般,2019,12,2019年12月,1,154,89,0,7957
3,202001,オンライン,一般,2020,1,2020年1月,0,703,55,6,2379
4,202001,店頭タブレット,ゴールド,2019,12,2019年12月,1,163,109,1,24668


In [139]:
#初回利用率
df_sql['rate_1st_use'] = df_sql['sum_no_1st_use']/df_sql['sum_no_account']

#クレジット初回利用単価
df_sql['unit_1st_amt'] = df_sql['sum_credit_amt6m']/df_sql['sum_no_1st_use']

#解約率
df_sql['rate_kaiyaku'] = df_sql['sum_no_kaiyaku']/df_sql['sum_no_account']


df_sql.head()

,cp,channel,card_cate,kanyu_year,kanyu_month,kanyu_yymm,mob,sum_no_account,sum_no_1st_use,sum_no_kaiyaku,sum_credit_amt6m,rate_1st_use,unit_1st_amt,rate_kaiyaku
0,202001,オンライン,ゴールド,2019,12,2019年12月,1,34,29,1,8145,0.852941,280.862069,0.029412
1,202001,オンライン,ゴールド,2020,1,2020年1月,0,29,13,0,1070,0.448276,82.307692,0.000000
2,202001,オンライン,一般,2019,12,2019年12月,1,154,89,0,7957,0.577922,89.404494,0.000000
3,202001,オンライン,一般,2020,1,2020年1月,0,703,55,6,2379,0.078236,43.254545,0.008535
4,202001,店頭タブレット,ゴールド,2019,12,2019年12月,1,163,109,1,24668,0.668712,226.311927,0.006135


CSVアウトプット

In [140]:
df_sql.to_csv(data_dir /r"Output"/f"{output_file1}_{input_file}.csv", index=None,header=True,encoding='utf-8-sig')

アフィリエイト別<br>積み残しメモ<br>アフィリエイトの累積承認リストを正しく作る必要あり

In [144]:
print(input_file2)

デジガレアフィリエイト申込累積


In [145]:
bk = pd.ExcelFile(data_dir2/r"input"/f"{input_file2}.xlsx")
print(bk.sheet_names)

['統合（通常）', '統合（利用条件付き）']


In [146]:
df1 = pd.read_excel(data_dir2 /r"input"/f"{input_file2}.xlsx", sheet_name=0)
df2 = pd.read_excel(data_dir2 /r"input"/f"{input_file2}.xlsx", sheet_name=1)
df_affi = pd.concat([df1,df2])
df_affi

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価
0,37,げん玉,ポイント,2020-01-08 18:43:29,2020-01-08 18:56:06,155051921643363,BGT,9000
1,37,げん玉,ポイント,2020-01-08 19:11:08,2020-01-08 19:32:05,155051921643397,BGT,9000
2,82,ちょびリッチ。ドットコム,ポイント,2020-01-08 20:07:13,2020-01-08 20:18:06,155051921643538,BGT,9000
3,52,ECナビSP,ポイント,2020-01-08 20:21:44,2020-01-08 20:31:05,155051921643553,BGT,9000
4,52,ECナビSP,ポイント,2020-01-08 20:31:50,2020-01-08 20:39:06,155051921643587,BGT,9000
...,...,...,...,...,...,...,...,...
1159,21,ポイントインカム,ポイント,2020-07-31 21:31:09,2020-07-31 21:54:08,155051921979171,BGT,9000
1160,0,ライフメディア,NaN,2020-05-02 00:00:00,2020-05-02 00:00:00,155051921871154,BGT,9000
1161,0,げん玉,NaN,NaT,2020-05-11 00:00:00,155051921883829,BGT,9000
1162,0,ポイントインカム,NaN,2020-05-11 08:59:46,2020-05-11 00:00:00,155051921883753,BGT,9000


In [147]:
df_affi = df_affi.rename(columns={'メディア名':'media',
                                  '媒体チャネル':'baitai',
                                  '参照用':'applino'})
df_affi = df_affi[['media','baitai','applino']]
df_affi.head(2)

,media,baitai,applino
0,げん玉,ポイント,155051921643363
1,げん玉,ポイント,155051921643397


In [148]:
q = """
        SELECT
        cp
        ,channel
        ,kanyu_year
        ,kanyu_month
        ,kanyu_yymm
        ,mob
        ,media
        ,baitai
        ,count(t1.applino) as sum_no_account
        ,sum(case when first_use_dt > 0 then 1 else 0 end) as sum_no_1st_use
        ,sum(case when kaiyaku_f = 1 then 1 else 0 end) as sum_no_kaiyaku
        ,sum(credit_amt_6m) as sum_credit_amt6m
        from 
            df_dup t1
        inner join
            df_affi t2
        on t1.applino = t2.applino
        group by
            1,2,3,4,5,6,7,8
        """

In [149]:
df_sql = sqldf(q, locals())

In [150]:
#初回利用率
df_sql['rate_1st_use'] = df_sql['sum_no_1st_use']/df_sql['sum_no_account']

#クレジット初回利用単価
df_sql['unit_1st_amt'] = df_sql['sum_credit_amt6m']/df_sql['sum_no_1st_use']

#解約率
df_sql['rate_kaiyaku'] = df_sql['sum_no_kaiyaku']/df_sql['sum_no_account']


df_sql.head()

,cp,channel,kanyu_year,kanyu_month,kanyu_yymm,mob,media,baitai,sum_no_account,sum_no_1st_use,sum_no_kaiyaku,sum_credit_amt6m,rate_1st_use,unit_1st_amt,rate_kaiyaku
0,202006,オンライン,2020,1,2020年1月,5,CLUBPanasonicコイン,ポイント,4,4,0,148,1.000000,37.0,0.000000
1,202006,オンライン,2020,1,2020年1月,5,ECナビPC,ポイント,7,2,2,16,0.285714,8.0,0.285714
2,202006,オンライン,2020,1,2020年1月,5,ECナビSP,ポイント,9,4,0,66,0.444444,16.5,0.000000
3,202006,オンライン,2020,1,2020年1月,5,GetMoney!(JIPC参加企業)（連携用）,ポイント,12,2,2,34,0.166667,17.0,0.166667
4,202006,オンライン,2020,1,2020年1月,5,NetMile（すぐたま）,ポイント,12,5,4,137,0.416667,27.4,0.333333


In [151]:
df_sql.head()

,cp,channel,kanyu_year,kanyu_month,kanyu_yymm,mob,media,baitai,sum_no_account,sum_no_1st_use,sum_no_kaiyaku,sum_credit_amt6m,rate_1st_use,unit_1st_amt,rate_kaiyaku
0,202006,オンライン,2020,1,2020年1月,5,CLUBPanasonicコイン,ポイント,4,4,0,148,1.000000,37.0,0.000000
1,202006,オンライン,2020,1,2020年1月,5,ECナビPC,ポイント,7,2,2,16,0.285714,8.0,0.285714
2,202006,オンライン,2020,1,2020年1月,5,ECナビSP,ポイント,9,4,0,66,0.444444,16.5,0.000000
3,202006,オンライン,2020,1,2020年1月,5,GetMoney!(JIPC参加企業)（連携用）,ポイント,12,2,2,34,0.166667,17.0,0.166667
4,202006,オンライン,2020,1,2020年1月,5,NetMile（すぐたま）,ポイント,12,5,4,137,0.416667,27.4,0.333333


In [152]:
df_sql.shape[0]

170

In [153]:
df_sql.to_csv(data_dir /r"Output"/f"{output_file2}_{input_file}.csv", index=None,header=True,encoding='utf-8-sig')